In [11]:
import numpy as np
from matplotlib import pyplot as plt
from patchify import patchify
import tifffile as tiff
from tensorflow.keras.callbacks import EarlyStopping
import os
from PIL import Image 
import numpy as np 
from patchify import patchify
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import metrics
from matplotlib import pyplot as plt
import random
from osgeo import gdal
import imageio
from keras.models import load_model
import os
from PIL import Image 
import numpy as np 
from patchify import patchify
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from osgeo import gdal
import imageio
from matplotlib import pyplot as plt
import random
from keras.optimizers import Adam
import segmentation_models as sm

In [13]:
# converts any image with more than 3 channels to a 3 channel image
Gtransform = 0 # this will be used to give  the georeferenccing information to the reconstructed mask image 
Ptransform = 0 #this will be used to give  the georeferenccing information to the reconstructed mask image 
def convert_to_rgb(filename):
    # Open the original image using GDAL
    src_ds = gdal.Open(filename)
    # Read the georeferencing information
    geotransform = src_ds.GetGeoTransform()
    projection = src_ds.GetProjection()
    Gtransform = src_ds.GetGeoTransform()
    Ptransform = src_ds.GetProjection()
    # Convert the image to RGB using PIL
    image = imageio.imread(filename)

    # Convert the image to RGB
    image = image[..., :3]

    # Save the image as a TIFF file
    imageio.imwrite(filename, image)

    # Open the converted image using GDAL
    dst_ds = gdal.Open(filename, gdal.GA_Update)

    # Write the georeferencing information to the converted image
    dst_ds.SetGeoTransform(geotransform)
    dst_ds.SetProjection(projection)

    # Close the datasets
    src_ds = None
    dst_ds = None

/Users/macbookair/anaconda3/envs/ML/lib/python3.10/site-packages/osgeo/gdal.py:287: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
/var/folders/ks/cpcnjkt95qx6v7hynw8xydkr0000gp/T/ipykernel_53190/788469906.py:12: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread('farm.tif')


In [ ]:
convert(filename)

In [14]:
large_image_stack = tiff.imread(filename)

In [15]:
patches_img = patchify(large_image_stack, (256, 256,3), step=256)
all_images_patches = []
for i in range(patches_img.shape[0]):
    for j in range(patches_img.shape[1]):
        single_patch_mask = patches_img[i,j,:,:]
        min_val = single_patch_mask.min()
        max_val = single_patch_mask.max()
        single_patch_mask = (single_patch_mask - min_val) / (max_val - min_val)
        single_patch_mask = single_patch_mask.astype('float32')
        single_patch_mask = single_patch_mask[0]
        all_images_patches.append(single_patch_mask)

In [17]:
model = load_model(#input_model_name_here, compile=False)
optimizer = Adam()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
image_dataset = np.array(all_images_patches)
predicted_classes = []
for i in range(0,len(image_dataset[:])):
    test_img = image_dataset[i]
    test_img_input=np.expand_dims(test_img, 0)
    prediction = model.predict(test_img_input)
    prediction = prediction[0,:,:,0]
    prediction = (prediction >= 0.5).astype(float)
    predicted_classes.append(prediction)

length = patches_img.shape[0] * patches_img.shape[3]
breath = patches_img.shape[1] * patches_img.shape[4]
predicted_classes_array =  np.array(predicted_classes)
predicted_classes_array = predicted_classes_array.reshape((patches_img.shape[0],patches_img.shape[1],patches_img.shape[3],patches_img.shape[4]))
reconstructed_image = unpatchify(predicted_classes_array, (length, breath))

(12, 41, 1, 256, 256, 3)


In [ ]:
Filename = #input your desired file name as a string

In [38]:
imageio.imsave(Filename, reconstructed_image)

In [ ]:
dst_ds = gdal.Open(Filename, gdal.GA_Update)
dst_ds.SetGeoTransform(geotransform)
dst_ds.SetProjection(projection)

# Close the datasets
src_ds = None
dst_ds = None